# does it make sense to ingnore the structure??

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')
import matplotlib.pyplot as plt

# Load data

In [ ]:
import ShaKer.rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()
dataflat = {k:(data[k][0],data[k][1],'.'*len(data[k][1]) ) for k in data}

print dataflat

# train predict

In [ ]:
import ShaKer.simushape as sim
from ShaKer.rna_tools.rnaplfold import rnaplfold


model  = sim.make_model(data,['23sRNA','R009'])
model2  = sim.make_model(dataflat,['23sRNA','R009'])

data.pop("23sRNA")
data.pop("R009")
dataflat.pop("23sRNA")
dataflat.pop("R009")



keys = data.keys()

def get_str(seq):
    return ['.'*len(seq)]

    
predictions =     { k: (data[k][1],sim.predict(model,data[k][1])) for k in keys  }
predictions_nostr = { k: (data[k][1],sim.predict(model2,data[k][1], seq_to_db_function=get_str)) for k in keys  }

acc_shaker = {k: rnaplfold(*predictions[k]) for  k in keys} 
acc_shaker_nostr = {k: rnaplfold(*predictions_nostr[k]) for  k in keys} 


In [ ]:
from scipy.stats import spearmanr as spear 

for k in keys:
    print "%s \t %.2f \t %.2f" % (k, spear(predictions[k][1],data[k][0])[0],spear(predictions_nostr[k][1],data[k][0])[0]    )

# real and sukosd based accessibility

In [ ]:
acc_shape =  {k: rnaplfold(data[k][1],data[k][0]) for  k in keys}  


In [ ]:

corr = lambda ac1, ac2:[spear(ac1[k], ac2[k])[0] for k in keys]

corr_shaker =  corr(acc_shaker, acc_shape)
corr_shaker_nostr =  corr(acc_shaker_nostr, acc_shape)


# Visualize 

In [ ]:
import pandas
import numpy as np
index= ['Shaker',"ShaKer_nostr"]

data2 = [corr_shaker,corr_shaker_nostr]


import seaborn as sns
sns.boxplot(data=pandas.DataFrame(data2, index=index).T)
TITLE = "Accessibility"
plt.title("SOME TITLE")
plt.grid()

---